# Google Sentiment Analysis API

In [1]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums
from google.cloud import storage
from time import sleep
import pandas as pd
import sys

pandas.options.mode.chained_assignment = None

ImportError: cannot import name 'storage' from 'google.cloud' (unknown location)

In [8]:
def parse_csv_from_gcs(csv_file, drop = 1):
    df = pd.read_csv(csv_file, encoding = "ISO-8859-1")
    df.fillna("", inplace=True)
    return df

# Aquí podriamos agregar una columna de estas por prueba (opción a)
# e.g. g_score_adjs_en, g_magnitude_adjs_en, etc
def add_score_magnitude_cols(dataframe):
    dataframe["g_score"] = ""
    dataframe["g_magnitude"] = ""
    
# o bien separar el dataframe con solo las columnas  (opción b)
# y a este nuevo df le agregamos las columnas de g_score y g_magnnitude
def select_cols(dataframe, language = 'es', char = 'complete'):
    if language == 'es':
        if char == 'complete':
            df = dataframe[['missing_info', 'improvements']]
            return df
        if char == 'adj':
            df = dataframe[['missing_info'+'_'+char, 'improvements'+'_'+char]]
            return df
        if char == 'noun_adj':
            df = dataframe[['missing_info'+'_'+char, 'improvements'+'_'+char]]
            return df
        
    if language == 'en':
        if char == 'complete':
            df = dataframe[['missing_info'+'_'+language, 'improvements'+'_'+language]]
            return df
        if char == 'adj':
            df = dataframe[['missing_info'+'_'+language+'_'+char, 'improvements'+'_'+language+'_'+char]]
            return df
        if char == 'noun_adj':
            df = dataframe[['missing_info'+'_'+language+'_'+char, 'improvements'+'_'+language+'_'+char]]
            return df
    
def analyze_sentiment(gcs_content_uri, client, language):
    """
    Analyzing Sentiment in text file stored in Cloud Storage
    Args:
      -gcs_content_uri Google Cloud Storage URI where the file content is located.
       e.g. gs://[Your Bucket]/[Path to File]
      -client:
       e.g. language_v1.LanguageServiceClient()
      -language: 'es' or 'en'
       For list of supported languages:
       https://cloud.google.com/natural-language/docs/languages
    """    
    if gcs_content_uri == "":
        return ""
    type_ = enums.Document.Type.PLAIN_TEXT
    document = {"content": gcs_content_uri, "type": type_, "language": language}
    encoding_type = enums.EncodingType.UTF8
    response = client.analyze_sentiment(document, encoding_type=encoding_type)
    return response

In [ ]:
# Sentiment analysis option a):
gcs_path = sys.argv[1]
output_bucket = sys.argv[2]
output_csv_file = sys.argv[3]

df = parse_csv_from_gcs(gcs_path)
add_score_magnitude_cols(df)
cols = ['missing_info', 'improvements', 'missing_info_en', 'improvements_en',
               'missing_info_adj', 'improvements_adj', 'missing_info_noun_adj', 
               'improvements_noun_adj', 'missing_info_en_adj','improvements_en_adj',
               'missing_info_en_noun_adj', 'improvements_en_noun_adj']

client = language_v1.LanguageServiceClient()
lang = 'es'

for i in df.index:
    response = analyze_sentiment(df.at[i, cols[1]], client, language = lang)
    df.at[i, 'g_core'] = response.document_sentiment.score
    df.at[i, 'g_magnitude'] = response.document_sentiment.magnitude
    sleep(0.5)

In [ ]:
# Sentiment analysis option b):

gcs_path = sys.argv[1]
output_bucket = sys.argv[2]
output_csv_file = sys.argv[3]

lang = 'es'

dataframe = parse_csv_from_gcs(gcs_path)
df = select_cols(dataframe, language = lang, char = 'complete')
add_score_magnitude_cols(df)
missing_info = df.columns[0]
improvements = df.columns[1]

client = language_v1.LanguageServiceClient()

for i in df.index:
    response = analyze_sentiment(df.at[i, missing_info], client,language = lang)
    df.at[i, 'g_core'] = response.document_sentiment.score
    df.at[i, 'g_magnitude'] = response.document_sentiment.magnitude
    sleep(0.5)

In [ ]:
# Export data
df.to_csv("survey_results.csv", encoding = 'ISO-8859-1')

gcs = storage.Client()
gcs.get_bucket(output_bucket).blob(output_csv_file).upload_from_filename('survey_results.csv', content_type='text/csv')